In [1]:
from utils import *
import tensorflow as tf
from keras import backend as K

2022-10-22 12:14:02.490216: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-22 12:14:02.600315: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-22 12:14:03.100708: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/moises/workspace/devel/lib
2022-10-22 12:14:03.100764: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.

In [2]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
K._get_available_gpus()

2022-10-22 12:14:03.614103: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-22 12:14:03.653785: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 12:14:03.673867: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 12:14:03.674036: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5



2022-10-22 12:14:04.042559: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 12:14:04.042752: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 12:14:04.042877: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 12:14:04.042973: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4005 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5
2022-10-22 12:14:04.044366: I tensorflow/stream_executor/cuda/cuda_gpu_executo

['/device:GPU:0']

In [3]:
tickers_sets_path = 'data/tickers/sets/'

sets_files = sorted([tickers_sets_path + x for x in listdir(tickers_sets_path) if 'json' in x], reverse=True)

tickers_sets = []

for set_file in sets_files:
    with open(set_file, 'r') as f:
        tickers_sets.append(json.load(f))

dst_path = 'data/stocks/'

In [4]:
df = pd.read_csv('data/dataset.csv')
df = df.set_index('Date')
df = df[:-80]

In [5]:
data_size = 4800
input_tickers = tickers_sets[0]
output_tickers = tickers_sets[0]

In [6]:
scaler, x, y, dic = dataa(df, data_size, input_tickers, output_tickers, step_size=1, input_size=60, output_size=20, feature_range=(0,1))

set()
394 394
394 394


In [7]:
x_train, y_train, x_val, y_val, x_test, y_test = train_val_test_split(x,y)

In [8]:
input_shape = (x.shape[1],x.shape[2])
output_shape = (y.shape[1], y.shape[2])
layers_info_list = [
    [
        (50, 0.2, True), 
        (70, 0.2, True),
        (70, 0.2, True),
        (60, 0.2, True),
        (40, 0.2, False)
    ],
    [
        (100, 0.2, True), 
        (100, 0.2, True),
        (100, 0.2, True),
        (150, 0.2, False)
    ],
    [
        (450, 0.2, True), 
        (600, 0.2, True),
        (600, 0.2, True),
        (400, 0.2, False)
    ],
    [
        (50, 0.2, True), 
        (50, 0.2, True),
        (50, 0.2, True),
        (50, 0.2, True),
        (50, 0.2, True),
        (50, 0.2, True),
        (50, 0.2, True),
        (50, 0.2, True),
        (50, 0.2, True),
        (50, 0.2, True),
        (50, 0.2, True),
        (50, 0.2, True),
        (50, 0.2, True),
        (50, 0.2, False)
    ],
    [
        (85, 0.2, True), 
        (85, 0.2, True),
        (85, 0.2, True),
        (85, 0.2, True),
        (85, 0.2, True),
        (85, 0.2, True),
        (85, 0.2, True),
        (85, 0.2, True),
        (85, 0.2, False)
    ]
]


In [9]:
"""
    [
        (600, 0.2, True), 
        (800, 0.2, True),
        (800, 0.2, True),
        (600, 0.2, False)
    ],"""

layers_info_list = [
    [
        (1000, 0.15, True), 
        (1000, 0.15, True),
        (1000, 0.15, False)
    ],
    [
        (800, 0.1, True), 
        (1000, 0.1, True),
        (1000, 0.1, True),
        (800, 0.1, False)
    ]
]

In [10]:
models = []

for layers_info in layers_info_list:

    models.append(create_model(input_shape, output_shape, layers_info, metrics=['accuracy', 'mse']))

In [11]:
history_list = []
epochs = 100

i = 5
set_name = 'set6/'
plot_path_dir = 'metrics/' + set_name

for model in models:
    
    try:
        
        history = model.fit(
            x=x_train,
            y=y_train,
            epochs=epochs,
            batch_size=32,
            validation_data=(x_val, y_val)
        )

        history_list.append(history)

        model_name = 'LSTM' + str(i)
        # model.save('models/' + set_name + model_name + '/model')

        plot_path_loss = plot_path_dir + 'LOSS_' + model_name + '.png'
        validation_plot(history.history['loss'], history.history['val_loss'], 'train vs validation loss', f=lambda: plt.savefig(plot_path_loss))

        plot_path_mse = plot_path_dir + 'MSE_' + model_name + '.png'
        validation_plot(history.history['mse'], history.history['val_mse'], 'train vs validation mse', f=lambda: plt.savefig(plot_path_mse), plot_type='mse')

        plot_path_accuracy = plot_path_dir + 'ACCURACY_' + model_name + '.png'
        validation_plot(history.history['accuracy'], history.history['val_accuracy'], 'train vs validation accuracy', f=lambda: plt.savefig(plot_path_accuracy), plot_type='accuracy')

    except:

        print('ERRO NO MODELO ' + str(i))
    
    
    i += 1

Epoch 1/100


2022-10-22 12:14:13.498174: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8600


104/104 [==============================] - 15s 115ms/step - loss: 0.1115 - accuracy: 0.0880 - mse: 0.0192 - val_loss: 0.0722 - val_accuracy: 0.2275 - val_mse: 0.0058
Epoch 2/100
104/104 [==============================] - 11s 108ms/step - loss: 0.0626 - accuracy: 0.2323 - mse: 0.0045 - val_loss: 0.0572 - val_accuracy: 0.2798 - val_mse: 0.0036
Epoch 3/100
104/104 [==============================] - 11s 109ms/step - loss: 0.0529 - accuracy: 0.2794 - mse: 0.0031 - val_loss: 0.0476 - val_accuracy: 0.3066 - val_mse: 0.0025
Epoch 4/100
104/104 [==============================] - 11s 109ms/step - loss: 0.0470 - accuracy: 0.3134 - mse: 0.0025 - val_loss: 0.0471 - val_accuracy: 0.2970 - val_mse: 0.0025
Epoch 5/100
104/104 [==============================] - 11s 109ms/step - loss: 0.0447 - accuracy: 0.3261 - mse: 0.0022 - val_loss: 0.0454 - val_accuracy: 0.3318 - val_mse: 0.0024
Epoch 6/100
104/104 [==============================] - 11s 109ms/step - loss: 0.0458 - accuracy: 0.3380 - mse: 0.0024 - va